In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

import intake

import config
import util

In [3]:
cat = intake.open_catalog("catalogs/fluxes-catalog-web.yml")
print(cat.yaml())

sources:
  fluxes-catalog-web:
    args:
      path: catalogs/fluxes-catalog-web.yml
    description: Flux datasets
    driver: intake.catalog.local.YAMLFileCatalog
    metadata: {}



In [4]:
ds = cat.fgco2_MPI_SOM_FFN.to_dask().compute()            
ds

<xarray.Dataset>
Dimensions:         (time: 432, lat: 180, lon: 360, bnds: 2)
Coordinates:
  * time            (time) datetime64[ns] 1982-01-15 1982-02-15 ... 2017-12-15
  * lat             (lat) float32 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon             (lon) float32 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
Dimensions without coordinates: bnds
Data variables: (12/14)
    spco2_raw       (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    fgco2_raw       (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    spco2_smoothed  (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    fgco2_smoothed  (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    sol             (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    kw              (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    ...              ...
    dco2_raw        (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    dco2_smoothed   (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    seamask         (lat, lon) int32 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    time_bnds       (time, bnds) datetime64[ns] 1981-12-31 ... 2017-12-31
    lat_bnds        (lat, bnds) float32 -90.0 -89.0 -89.0 ... 89.0 89.0 90.0
    lon_bnds        (lon, bnds) float32 -180.0 -179.0 -179.0 ... 179.0 180.0
Attributes:
    institution:    MPI-MET, Hamburg, Germany (former: ETH Zurich, Switzerland)
    institude_id:   MPI
    model_id:       SOM-FFN
    run_id:         v2018
    contact:        Peter Landschutzer (peter.landschuetzer@mpimet.mpg.de)
    creation_date:  2019-03-21

In [5]:
ds['area'] = util.compute_grid_area(ds)

In [6]:
ds = cat.fgapo_CarbonScope_apo99_v2020.to_dask().compute()            
ds

<xarray.Dataset>
Dimensions:        (lon: 144, nv: 2, lat: 90, itime: 11323, proc: 8, rt: 3, spec: 1, mtime: 7670, mtime0: 100)
Coordinates:
  * lon            (lon) float32 -178.8 -176.2 -173.8 ... 173.8 176.2 178.8
  * lat            (lat) float32 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
  * itime          (itime) int32 -315532800 -315446400 ... 662601600 662688000
  * proc           (proc) int32 1 2 3 4 5 6 7 8
  * rt             (rt) int32 1 2 3
  * spec           (spec) int32 1
  * mtime          (mtime) int32 -31492801 -31406401 ... 631022400 631108800
Dimensions without coordinates: nv, mtime0
Data variables: (12/16)
    lon_bounds     (lon, nv) float32 -180.0 -177.5 -177.5 ... 177.5 177.5 180.0
    lat_bounds     (lat, nv) float32 -90.0 -88.0 -88.0 -86.0 ... 88.0 88.0 90.0
    itime_bounds   (itime, nv) int32 -315532800 -315446400 ... 662774400
    dt             (itime) int32 86400 86400 86400 86400 ... 86400 86400 86400
    year           (itime) float64 1.99e+03 1.99e+03 ... 2.021e+03 2.021e+03
    lproc          (proc) |S20 b'ocL                 ' ... b'ini             ...
    ...             ...
    myear          (mtime) float64 1.999e+03 1.999e+03 ... 2.02e+03 2.02e+03
    tmask          (itime, mtime0) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    apoflux_land   (mtime, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    apoflux_ocean  (mtime, lat, lon) float32 0.0 0.0 0.0 ... 0.0009978 0.0009971
    apoflux_subt   (mtime, lat, lon) float32 0.0 0.0 0.0 ... 1.765e-05 2.051e-05
    apoflux_excl   (mtime, lat, lon) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes: (12/14)
    title:         Jena CarboScope results
    history:       Inversion code by Christian Roedenbeck
    institution:   MPI Biogeochemistry Jena
    references:    http://www.BGC-Jena.mpg.de/CarboScope/
    Conventions:   CF-1.1
    species:       apo
    ...            ...
    valid_period:  1999-2019 (inclusive)
    yrfi_valid:    1999
    yrfe_valid:    2019
    dirname:       OUTPUT13.033+tm3_ncep_fg.1990-2020_2Hx2.apo.-S-.apo97A+fbS...
    filename:      mu1.0_090_flux_asis+1999-2019.nc
    c0:            -18.0

In [7]:
cluster, client = util.get_ClusterClient()
cluster.adapt(minimum_jobs=2, maximum_jobs=5)
cluster

In [8]:
%%time
def retrieve_dataset(value):
    try:
        ds = cat.fgco2_fff_odiac(year=value.year, month=value.month, day=value.day).to_dask()
    except Exception:
        ds = None
        raise
    return ds


date_range = pd.date_range(start='2000-01-01', end='2019-01-01')
ds_list = client.map(retrieve_dataset, date_range)
ds_list = client.gather(ds_list)
#ds_list = [ds for ds in ds_list if ds is not None]
#ds = xr.concat(ds_list, dim='time', compat='override', coords='minimal').squeeze()
ds_list

CPU times: user 20.5 s, sys: 1.49 s, total: 22 s
Wall time: 1min 54s


[<xarray.Dataset>
 Dimensions:                (n_hour: 24, n_lat: 180, n_lon: 360, n_itime: 6)
 Dimensions without coordinates: n_hour, n_lat, n_lon, n_itime
 Data variables:
     emission               (n_hour, n_lat, n_lon) float64 dask.array<chunksize=(24, 180, 360), meta=np.ndarray>
     surface_area           (n_lat, n_lon) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
     hourly_total_emission  (n_hour) float64 dask.array<chunksize=(24,), meta=np.ndarray>
     itime                  (n_hour, n_itime) int16 dask.array<chunksize=(24, 6), meta=np.ndarray>
 Attributes:
     total_emis:         18.27 TgC
     odiac_total:  Original ODIAC emission for this month =       18.94 TgC/day,
 <xarray.Dataset>
 Dimensions:                (n_hour: 24, n_lat: 180, n_lon: 360, n_itime: 6)
 Dimensions without coordinates: n_hour, n_lat, n_lon, n_itime
 Data variables:
     emission               (n_hour, n_lat, n_lon) float64 dask.array<chunksize=(24, 180, 360), meta=np.ndarray>
     